# Amazon Bedrock AgentCore를 활용한 Production 배포

이 노트북에서는 `01_external_dbs.ipynb`에서 구현한 MCP 기반 Agent를 Amazon Bedrock AgentCore를 사용하여 프로덕션 환경에 배포하는 방법을 다룹니다.

## 학습 목표
- Amazon Bedrock AgentCore의 핵심 구성 요소 이해
- Strands Agents를 AgentCore Runtime에 배포
- MCP 도구를 AgentCore Gateway를 통해 연동
- Production 환경에서의 모니터링 및 관리

## Amazon Bedrock AgentCore 소개

2025년 7월에 출시된 Amazon Bedrock AgentCore는 AI 에이전트를 안전하고 확장 가능하게 배포하고 운영할 수 있는 서비스입니다.

### 주요 구성 요소:
- **Runtime**: 서버리스 런타임으로 다양한 프레임워크 지원
- **Gateway**: API와 서비스를 MCP 호환 도구로 변환
- **Memory**: 완전 관리형 메모리 인프라
- **Identity**: AWS 서비스 전반의 ID 및 액세스 관리

## 1. 환경 설정

Amazon Bedrock AgentCore를 사용하기 위한 환경을 설정합니다.

In [ ]:
# 필요한 패키지 설치
!pip install bedrock-agentcore-starter-toolkit strands-agents boto3 mcp arxiv chembl-webresource-client --quiet

In [ ]:
# 라이브러리 임포트
import os
import json
import logging
from typing import Dict, Any, List, Optional
from datetime import datetime

# AWS SDK
import boto3
from botocore.config import Config

# Bedrock AgentCore
from bedrock_agentcore import (
    BedrockAgentCoreApp,
    AgentCoreConfig,
    RuntimeConfig,
    GatewayConfig,
    MemoryConfig,
    IdentityConfig
)

# Strands Agents
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient

# MCP
from mcp import stdio_client, StdioServerParameters
from mcp.server.fastmcp import FastMCP

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("agentcore_production")

## 2. AgentCore 애플리케이션 구성

HER2 연구를 위한 통합 Agent를 AgentCore 애플리케이션으로 구성합니다.

In [ ]:
# AgentCore 애플리케이션 생성
app = BedrockAgentCoreApp(
    name="her2-research-agent",
    description="HER2 양성 암 연구를 위한 통합 AI Agent"
)

# AgentCore 구성 설정
config = AgentCoreConfig(
    runtime=RuntimeConfig(
        memory_size=2048,  # MB
        timeout=300,  # seconds
        environment_variables={
            "LOG_LEVEL": "INFO",
            "MAX_RETRIES": "3"
        }
    ),
    gateway=GatewayConfig(
        enable_mcp=True,
        allowed_origins=["*"],
        rate_limit=100  # requests per minute
    ),
    memory=MemoryConfig(
        enable_session_memory=True,
        ttl_seconds=3600,  # 1 hour
        max_sessions=1000
    ),
    identity=IdentityConfig(
        enable_user_context=True,
        authentication_required=True
    )
)

app.configure(config)

## 3. MCP 도구 통합

Arxiv와 ChEMBL MCP 서버를 AgentCore Gateway를 통해 통합합니다.

In [ ]:
%%writefile mcp_arxiv_agentcore.py
"""
Arxiv MCP Server for AgentCore
"""
from mcp.server.fastmcp import FastMCP
import arxiv
import logging
from typing import Dict, Any, List
from datetime import datetime, timezone
from dateutil import parser

# FastMCP 초기화
mcp = FastMCP(name="arxiv_tools")
logger = logging.getLogger("arxiv_mcp")

MAX_RESULTS = 10

def is_within_date_range(
    date: datetime, start: datetime | None, end: datetime | None
) -> bool:
    """날짜가 지정된 범위 내에 있는지 확인"""
    if start and not start.tzinfo:
        start = start.replace(tzinfo=timezone.utc)
    if end and not end.tzinfo:
        end = end.replace(tzinfo=timezone.utc)
    
    if start and date < start:
        return False
    if end and date > end:
        return False
    return True

def process_paper(paper: arxiv.Result) -> Dict[str, Any]:
    """논문 정보를 처리하여 표준 형식으로 반환"""
    return {
        "id": paper.get_short_id(),
        "title": paper.title,
        "authors": [author.name for author in paper.authors],
        "abstract": paper.summary,
        "categories": paper.categories,
        "published": paper.published.isoformat(),
        "url": paper.pdf_url,
        "resource_uri": f"arxiv://{paper.get_short_id()}",
    }

@mcp.tool()
async def search_papers(
    query: str,
    max_results: int = 10,
    date_from: str = None,
    date_to: str = None,
    categories: List[str] = None
) -> List[Dict[str, Any]]:
    """Arxiv에서 논문을 검색합니다.
    
    Args:
        query: 검색 쿼리
        max_results: 최대 결과 수
        date_from: 시작 날짜 (YYYY-MM-DD)
        date_to: 종료 날짜 (YYYY-MM-DD)
        categories: 카테고리 필터
    
    Returns:
        검색된 논문 목록
    """
    try:
        client = arxiv.Client()
        max_results = min(int(max_results), MAX_RESULTS)
        
        # 카테고리 필터링이 있는 경우 쿼리 구성
        if categories:
            category_filter = " OR ".join(f"cat:{cat}" for cat in categories)
            query = f"({query}) AND ({category_filter})"
        
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.SubmittedDate,
        )
        
        # 날짜 필터링과 함께 결과 처리
        results = []
        date_from_obj = parser.parse(date_from).replace(tzinfo=timezone.utc) if date_from else None
        date_to_obj = parser.parse(date_to).replace(tzinfo=timezone.utc) if date_to else None
        
        for paper in client.results(search):
            if is_within_date_range(paper.published, date_from_obj, date_to_obj):
                results.append(process_paper(paper))
            if len(results) >= max_results:
                break
        
        return results
    
    except Exception as e:
        logger.error(f"Search error: {str(e)}")
        return [{"error": f"Search failed: {str(e)}"}]

@mcp.tool()
async def get_paper_details(paper_id: str) -> Dict[str, Any]:
    """특정 논문의 상세 정보를 가져옵니다.
    
    Args:
        paper_id: Arxiv 논문 ID
    
    Returns:
        논문 상세 정보
    """
    try:
        client = arxiv.Client()
        search = arxiv.Search(id_list=[paper_id])
        
        for paper in client.results(search):
            return process_paper(paper)
        
        return {"error": f"Paper with ID {paper_id} not found"}
    
    except Exception as e:
        logger.error(f"Get paper error: {str(e)}")
        return {"error": f"Failed to get paper: {str(e)}"}

if __name__ == "__main__":
    mcp.run()

In [ ]:
%%writefile mcp_chembl_agentcore.py
"""
ChEMBL MCP Server for AgentCore
"""
from mcp.server.fastmcp import FastMCP
import logging
from typing import Any, List, Dict
from chembl_webresource_client.new_client import new_client

# FastMCP 초기화
mcp = FastMCP(name="chembl_tools")
logger = logging.getLogger("chembl_mcp")

MAX_ACTIVITY_RESULTS = 100

@mcp.tool()
async def get_compound_activity(compound_name: str) -> List[Dict[str, Any]]:
    """특정 화합물의 활성 데이터를 가져옵니다.
    
    Args:
        compound_name: 화합물 이름
    
    Returns:
        활성 데이터 목록
    """
    try:
        client = new_client
        molecule = client.molecule.filter(
            pref_name__iexact=compound_name
        ).only('molecule_chembl_id')
        
        if not molecule:
            return [{"error": f"Compound {compound_name} not found"}]
        
        molecule_id = molecule[0]['molecule_chembl_id']
        
        activity = list(
            client.activity
            .filter(molecule_chembl_id=molecule_id)
            .filter(standard_type="IC50")
            .only(['pchembl_value', 'assay_description', 'canonical_smiles'])
        )
        
        if len(activity) > MAX_ACTIVITY_RESULTS:
            activity = activity[:MAX_ACTIVITY_RESULTS]
        
        return activity
    
    except Exception as e:
        logger.error(f"Get compound activity error: {str(e)}")
        return [{"error": f"Failed to get compound activity: {str(e)}"}]

@mcp.tool()
async def get_target_activity(target_name: str) -> List[Dict[str, Any]]:
    """특정 타겟의 활성 데이터를 가져옵니다.
    
    Args:
        target_name: 타겟 이름 (예: HER2, EGFR)
    
    Returns:
        활성 데이터 목록
    """
    try:
        client = new_client
        target = client.target.filter(
            target_synonym__icontains=target_name,
            organism='Homo sapiens'
        ).only('target_chembl_id')
        
        if not target:
            return [{"error": f"Target {target_name} not found"}]
        
        target_id = target[0]['target_chembl_id']
        
        activity = list(
            client.activity
            .filter(target_chembl_id=target_id)
            .filter(standard_type="IC50")
            .only(['pchembl_value', 'assay_description', 'canonical_smiles', 'molecule_chembl_id'])
        )
        
        if len(activity) > MAX_ACTIVITY_RESULTS:
            activity = activity[:MAX_ACTIVITY_RESULTS]
        
        return activity
    
    except Exception as e:
        logger.error(f"Get target activity error: {str(e)}")
        return [{"error": f"Failed to get target activity: {str(e)}"}]

@mcp.tool()
async def search_compounds(
    query: str,
    max_results: int = 20
) -> List[Dict[str, Any]]:
    """화합물을 검색합니다.
    
    Args:
        query: 검색 쿼리
        max_results: 최대 결과 수
    
    Returns:
        화합물 목록
    """
    try:
        client = new_client
        molecules = list(
            client.molecule
            .filter(pref_name__icontains=query)
            .only(['molecule_chembl_id', 'pref_name', 'molecule_type', 'max_phase'])
        )[:max_results]
        
        return molecules
    
    except Exception as e:
        logger.error(f"Search compounds error: {str(e)}")
        return [{"error": f"Failed to search compounds: {str(e)}"}]

if __name__ == "__main__":
    mcp.run()

## 4. Agent 구현

HER2 연구를 위한 통합 Agent를 구현합니다.

In [ ]:
%%writefile her2_research_agent.py
"""
HER2 Research Agent for Amazon Bedrock AgentCore
"""
import json
import logging
from typing import Dict, Any, List, Optional
from datetime import datetime

from bedrock_agentcore import BedrockAgentCoreApp
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp import stdio_client, StdioServerParameters
from botocore.config import Config

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# AgentCore 앱 초기화
app = BedrockAgentCoreApp()

# Bedrock 모델 설정
def get_bedrock_model():
    """Amazon Bedrock 모델을 설정합니다."""
    return BedrockModel(
        boto_client_config=Config(
            read_timeout=600,
            connect_timeout=600,
            retries=dict(max_attempts=3, mode="adaptive"),
        ),
        model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=4096,
        temperature=0.1,
        top_p=0.9,
    )

# Agent 시스템 프롬프트
SYSTEM_PROMPT = """
당신은 HER2 양성 암 연구를 전문으로 하는 수석 연구원입니다.

다음 데이터베이스에 접근할 수 있습니다:
1. Arxiv: 최신 과학 논문과 연구 동향
2. ChEMBL: 화합물, 타겟, 생물학적 활성 데이터

연구 질문에 답할 때:
- Arxiv에서 최신 연구 동향과 논문을 검색하세요
- ChEMBL에서 관련 화합물과 활성 데이터를 조사하세요
- 두 소스의 정보를 통합하여 포괄적인 답변을 제공하세요
- 항상 출처를 명시하고 데이터를 구체적으로 제시하세요
- 과학적 근거와 함께 실용적인 인사이트를 제공하세요
"""

class HER2ResearchAgent:
    """HER2 연구를 위한 통합 Agent"""
    
    def __init__(self):
        self.model = get_bedrock_model()
        self.agent = None
        self._initialize_agent()
    
    def _initialize_agent(self):
        """Agent와 MCP 도구를 초기화합니다."""
        try:
            tools = []
            
            # Arxiv MCP 클라이언트
            arxiv_client = MCPClient(
                lambda: stdio_client(
                    StdioServerParameters(
                        command="python",
                        args=["mcp_arxiv_agentcore.py"]
                    )
                )
            )
            
            # ChEMBL MCP 클라이언트
            chembl_client = MCPClient(
                lambda: stdio_client(
                    StdioServerParameters(
                        command="python",
                        args=["mcp_chembl_agentcore.py"]
                    )
                )
            )
            
            # 도구 수집
            with arxiv_client as ax_client, chembl_client as ch_client:
                tools.extend(ax_client.list_tools_sync())
                tools.extend(ch_client.list_tools_sync())
            
            # Agent 생성
            self.agent = Agent(
                model=self.model,
                system_prompt=SYSTEM_PROMPT,
                tools=tools
            )
            
            logger.info(f"Agent initialized with {len(tools)} tools")
        
        except Exception as e:
            logger.error(f"Failed to initialize agent: {str(e)}")
            raise
    
    def process_request(self, request: Dict[str, Any]) -> Dict[str, Any]:
        """사용자 요청을 처리합니다."""
        try:
            # 요청에서 정보 추출
            user_message = request.get("message", "")
            session_id = request.get("session_id")
            user_context = request.get("user_context", {})
            
            # 로깅
            logger.info(f"Processing request - Session: {session_id}")
            
            # Agent 실행
            response = self.agent(user_message)
            
            # 응답 구성
            result = {
                "success": True,
                "message": response.message if hasattr(response, 'message') else str(response),
                "session_id": session_id,
                "timestamp": datetime.utcnow().isoformat(),
                "metadata": {
                    "model_used": self.model.model_id,
                    "tools_available": len(self.agent.tools) if self.agent else 0
                }
            }
            
            return result
        
        except Exception as e:
            logger.error(f"Error processing request: {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "session_id": session_id,
                "timestamp": datetime.utcnow().isoformat()
            }

# 전역 Agent 인스턴스
research_agent = HER2ResearchAgent()

@app.entrypoint
def invoke(payload: Dict[str, Any]) -> Dict[str, Any]:
    """AgentCore 엔트리포인트 함수
    
    Args:
        payload: 요청 페이로드
    
    Returns:
        처리 결과
    """
    return research_agent.process_request(payload)

@app.health_check
def health_check() -> Dict[str, Any]:
    """헬스 체크 엔드포인트"""
    return {
        "status": "healthy",
        "timestamp": datetime.utcnow().isoformat(),
        "agent_ready": research_agent.agent is not None
    }

@app.on_startup
async def startup():
    """애플리케이션 시작 시 실행"""
    logger.info("HER2 Research Agent starting up...")

@app.on_shutdown
async def shutdown():
    """애플리케이션 종료 시 실행"""
    logger.info("HER2 Research Agent shutting down...")

if __name__ == "__main__":
    # 로컬 테스트용
    app.run()

## 5. AgentCore 배포 구성

AgentCore에 배포하기 위한 구성 파일을 생성합니다.

In [ ]:
%%writefile .bedrock_agentcore.yaml
# Amazon Bedrock AgentCore 배포 구성
name: her2-research-agent
description: HER2 양성 암 연구를 위한 통합 AI Agent

# Runtime 설정
runtime:
  handler: her2_research_agent.py
  memory: 2048  # MB
  timeout: 300  # seconds
  architecture: arm64
  environment:
    LOG_LEVEL: INFO
    MAX_RETRIES: "3"
    BEDROCK_REGION: us-west-2

# Gateway 설정
gateway:
  enable_mcp: true
  mcp_servers:
    - name: arxiv
      command: python mcp_arxiv_agentcore.py
      description: Arxiv 논문 검색 도구
    - name: chembl
      command: python mcp_chembl_agentcore.py
      description: ChEMBL 화합물 데이터베이스 도구
  cors:
    allowed_origins:
      - "*"
    allowed_methods:
      - GET
      - POST
    allowed_headers:
      - Content-Type
      - Authorization
  rate_limit:
    requests_per_minute: 100
    burst_size: 10

# Memory 설정
memory:
  enable_session: true
  session_ttl: 3600  # 1 hour
  max_sessions: 1000
  storage_type: dynamodb
  table_name: her2-agent-sessions

# Identity 설정
identity:
  authentication: iam
  enable_user_context: true
  allowed_principals:
    - arn:aws:iam::*:role/BedrockAgentCoreExecutionRole

# Monitoring 설정
monitoring:
  enable_cloudwatch: true
  log_group: /aws/bedrock/agentcore/her2-research
  metrics_namespace: BedrockAgentCore/HER2Research
  enable_xray: true

# Auto-scaling 설정
scaling:
  min_instances: 1
  max_instances: 10
  target_cpu_utilization: 70
  scale_in_cooldown: 60
  scale_out_cooldown: 30

# Tags
tags:
  Environment: production
  Project: HER2Research
  Team: AIResearch
  ManagedBy: AgentCore

In [ ]:
%%writefile requirements.txt
# Amazon Bedrock AgentCore 요구사항
bedrock-agentcore-starter-toolkit>=1.0.0
strands-agents>=0.5.0
boto3>=1.34.0
mcp>=0.1.0
arxiv>=2.1.0
chembl-webresource-client>=0.10.8
python-dateutil>=2.8.2

## 6. 배포 및 테스트

AgentCore에 Agent를 배포하고 테스트합니다.

In [ ]:
# AgentCore CLI를 사용한 배포
!agentcore configure -e her2_research_agent.py

In [ ]:
# Agent 배포
!agentcore launch --config .bedrock_agentcore.yaml

In [ ]:
# 배포 상태 확인
!agentcore status her2-research-agent

## 7. Production 테스트

배포된 Agent를 테스트합니다.

In [ ]:
import requests
import json

# AgentCore 엔드포인트 (배포 후 제공됨)
AGENTCORE_ENDPOINT = "https://your-agent-id.agentcore.bedrock.amazonaws.com/invoke"

def test_agent(message: str):
    """배포된 Agent를 테스트합니다."""
    
    payload = {
        "message": message,
        "session_id": "test-session-001",
        "user_context": {
            "user_id": "researcher-001",
            "role": "senior_researcher"
        }
    }
    
    response = requests.post(
        AGENTCORE_ENDPOINT,
        json=payload,
        headers={
            "Content-Type": "application/json",
            "Authorization": "Bearer YOUR_TOKEN"  # IAM 인증 토큰
        }
    )
    
    if response.status_code == 200:
        result = response.json()
        print("✅ 성공적인 응답:")
        print(json.dumps(result, indent=2, ensure_ascii=False))
    else:
        print(f"❌ 오류 발생: {response.status_code}")
        print(response.text)

# 테스트 실행
test_message = "HER2 양성 유방암의 최신 치료법에 대해 조사해줘"
print(f"📝 테스트 메시지: {test_message}")
print("=" * 80)

# test_agent(test_message)  # 실제 배포 후 주석 해제

In [ ]:
# AgentCore CLI를 사용한 직접 테스트
test_payload = {
    "message": "HER2 관련 최신 연구 논문과 주요 화합물을 조사해줘",
    "session_id": "test-001"
}

# 로컬 테스트
!agentcore invoke '{json.dumps(test_payload)}'

## 8. 모니터링 및 관리

AgentCore의 모니터링 기능을 설정하고 사용합니다.

In [ ]:
import boto3
from datetime import datetime, timedelta

# CloudWatch 클라이언트
cloudwatch = boto3.client('cloudwatch', region_name='us-west-2')

def get_agent_metrics(agent_name: str, hours: int = 24):
    """Agent의 CloudWatch 메트릭을 가져옵니다."""
    
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(hours=hours)
    
    # 요청 수 메트릭
    response = cloudwatch.get_metric_statistics(
        Namespace='BedrockAgentCore/HER2Research',
        MetricName='InvocationCount',
        Dimensions=[
            {'Name': 'AgentName', 'Value': agent_name}
        ],
        StartTime=start_time,
        EndTime=end_time,
        Period=3600,  # 1시간 단위
        Statistics=['Sum', 'Average']
    )
    
    print(f"📊 {agent_name} 메트릭 ({hours}시간)")
    print("=" * 50)
    
    if response['Datapoints']:
        for dp in sorted(response['Datapoints'], key=lambda x: x['Timestamp']):
            print(f"시간: {dp['Timestamp']}")
            print(f"  요청 수: {dp.get('Sum', 0):.0f}")
            print(f"  평균 응답 시간: {dp.get('Average', 0):.2f}ms")
    else:
        print("데이터가 없습니다.")

# 메트릭 조회
# get_agent_metrics('her2-research-agent', hours=24)

In [ ]:
# 로그 조회
logs_client = boto3.client('logs', region_name='us-west-2')

def get_agent_logs(log_group: str, hours: int = 1):
    """Agent의 CloudWatch 로그를 조회합니다."""
    
    end_time = int(datetime.utcnow().timestamp() * 1000)
    start_time = end_time - (hours * 3600 * 1000)
    
    try:
        response = logs_client.filter_log_events(
            logGroupName=log_group,
            startTime=start_time,
            endTime=end_time,
            limit=50
        )
        
        print(f"📜 최근 {hours}시간 로그")
        print("=" * 50)
        
        for event in response['events']:
            timestamp = datetime.fromtimestamp(event['timestamp'] / 1000)
            print(f"[{timestamp}] {event['message']}")
    
    except Exception as e:
        print(f"로그 조회 오류: {e}")

# 로그 조회
# get_agent_logs('/aws/bedrock/agentcore/her2-research', hours=1)

## 9. 고급 기능

### 9.1 Session Memory 활용

In [ ]:
class SessionMemoryManager:
    """AgentCore Session Memory 관리"""
    
    def __init__(self, table_name: str = "her2-agent-sessions"):
        self.dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
        self.table = self.dynamodb.Table(table_name)
    
    def get_session(self, session_id: str) -> Dict[str, Any]:
        """세션 정보를 가져옵니다."""
        try:
            response = self.table.get_item(
                Key={'session_id': session_id}
            )
            return response.get('Item', {})
        except Exception as e:
            logger.error(f"Failed to get session: {e}")
            return {}
    
    def update_session(self, session_id: str, data: Dict[str, Any]):
        """세션 정보를 업데이트합니다."""
        try:
            self.table.put_item(
                Item={
                    'session_id': session_id,
                    'updated_at': datetime.utcnow().isoformat(),
                    **data
                }
            )
        except Exception as e:
            logger.error(f"Failed to update session: {e}")

# 사용 예시
memory_manager = SessionMemoryManager()

# 세션 정보 저장
memory_manager.update_session(
    "test-session-001",
    {
        "user_id": "researcher-001",
        "research_topic": "HER2 positive breast cancer",
        "papers_reviewed": ["arxiv:2401.12345", "arxiv:2401.67890"],
        "compounds_analyzed": ["Trastuzumab", "Pertuzumab"]
    }
)

### 9.2 비동기 처리 및 배치 작업

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor

class BatchProcessor:
    """배치 처리를 위한 클래스"""
    
    def __init__(self, agent_endpoint: str):
        self.endpoint = agent_endpoint
        self.executor = ThreadPoolExecutor(max_workers=10)
    
    async def process_batch(self, queries: List[str]) -> List[Dict[str, Any]]:
        """여러 쿼리를 동시에 처리합니다."""
        tasks = []
        
        for i, query in enumerate(queries):
            payload = {
                "message": query,
                "session_id": f"batch-{i}",
                "batch_mode": True
            }
            tasks.append(self._invoke_agent(payload))
        
        results = await asyncio.gather(*tasks)
        return results
    
    async def _invoke_agent(self, payload: Dict[str, Any]) -> Dict[str, Any]:
        """Agent를 비동기적으로 호출합니다."""
        loop = asyncio.get_event_loop()
        return await loop.run_in_executor(
            self.executor,
            self._sync_invoke,
            payload
        )
    
    def _sync_invoke(self, payload: Dict[str, Any]) -> Dict[str, Any]:
        """동기적으로 Agent를 호출합니다."""
        response = requests.post(
            self.endpoint,
            json=payload,
            headers={"Content-Type": "application/json"}
        )
        return response.json()

# 배치 처리 예시
batch_queries = [
    "HER2 양성 유방암의 최신 치료법",
    "Trastuzumab의 작용 메커니즘",
    "HER2 억제제의 내성 극복 전략",
    "차세대 HER2 표적 치료제"
]

# processor = BatchProcessor(AGENTCORE_ENDPOINT)
# results = await processor.process_batch(batch_queries)

## 10. 정리 및 다음 단계

### 이 노트북에서 학습한 내용

✅ **Amazon Bedrock AgentCore 구성 요소**
- Runtime: 서버리스 실행 환경
- Gateway: MCP 도구 통합
- Memory: 세션 관리
- Identity: 인증 및 권한 관리

✅ **Production 배포 프로세스**
1. MCP 서버 구현
2. Strands Agent 구성
3. AgentCore 앱 개발
4. 배포 구성 파일 작성
5. 배포 및 테스트

✅ **모니터링 및 관리**
- CloudWatch 메트릭
- 로그 분석
- 세션 메모리 관리

### Production 체크리스트

- [ ] IAM 역할 및 정책 설정
- [ ] VPC 및 보안 그룹 구성
- [ ] KMS 키 암호화 설정
- [ ] CloudWatch 알람 구성
- [ ] 백업 및 복구 전략
- [ ] 부하 테스트 수행
- [ ] 비용 최적화 검토

### 다음 단계

1. **확장 기능 구현**
   - 추가 데이터베이스 통합 (PubMed, ClinicalTrials.gov)
   - 고급 분석 도구 추가
   - 시각화 기능 구현

2. **성능 최적화**
   - 캐싱 전략 구현
   - 비동기 처리 최적화
   - 모델 추론 성능 개선

3. **거버넌스 강화**
   - 감사 로그 구현
   - 컴플라이언스 체크
   - 데이터 보호 강화